### PART 1 ###

# Segmenting and Clustering Neighbourhoods in Toronto

In [13]:
!pip install bs4

     |████████████████████████████████| 115 kB 2.6 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=e390311f04798a53a6942c60c083e07adaf375a3f6f2e294efee933361b65fde
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [14]:
import urllib.request
import pandas as pd
import requests
from bs4 import BeautifulSoup
#get the data and create the first dataframe
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [15]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [17]:
df.shape

(103, 3)

In [22]:
df.groupby(['PostalCode']).first()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


In [23]:
len(df['PostalCode'].unique())

103

In [24]:
df[df['Borough'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [25]:
df.shape

(103, 3)

### PART 2 ###

In [26]:
pip install geocoder

     |████████████████████████████████| 98 kB 2.4 MB/s eta 0:00:011
     |████████████████████████████████| 829 kB 6.1 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=2ed6aa84167b7c1554a8a1ecead9801c4cd1a7763f7b3a9758ef1c5d00f191c0
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [27]:
import geocoder

In [32]:
geo = pd.read_csv('Geospatial_Coordinates.csv', encoding = 'utf-8')
geo_new = geo.rename(columns={'Postal Code': 'PostalCode'})
geo_new.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# checking dtypes

In [34]:
geo.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [35]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [36]:
df.shape

(103, 3)

In [37]:
geo.shape

(103, 3)

### merge the df's and cleaning

In [41]:
geo_merge = pd.merge(df, geo_new, on="PostalCode")
geo_merge

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
